<a href="https://colab.research.google.com/github/jhighllight/programmers_AI/blob/main/2%EC%9D%BC%EC%B0%A8_%EC%8B%A4%EC%8A%B5_%EC%95%A0%ED%94%8C_%EC%A3%BC%EC%8B%9D_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%B2%98%EB%A6%AC%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

애플 주식 데이터를 가지고 간단한 데이터 분석을 해보자. 모든 답은 Pyspark을 통해 이뤄져야 한다.

먼저 PySpark과 Py4J를 설치하자

In [ ]:
!pip install pyspark==3.0.1 py4j==0.10.9

#### Spark Session 만들기

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark Dataframe basic example") \
    .getOrCreate()

#### 애플 주식 CSV 파일 로딩하기: https://pyspark-test-sj.s3-us-west-2.amazonaws.com/appl_stock.csv
일단 pandas 데이터프레임으로 로딩해서 Spark 데이터프레임으로 변경한다

In [ ]:
import pandas as pd

apple_pandas_df = pd.read_csv("https://pyspark-test-sj.s3-us-west-2.amazonaws.com/appl_stock.csv")
apple_spark_df = spark.createDataFrame(apple_pandas_df)

#### 1> 어떤 컬럼 이름들이 있는가?

In [ ]:
apple_spark_df.columns

#### 2> 스키마를 프린트해보기

In [ ]:
apple_spark_df.printSchema()

#### 3> 처음 5개의 레코드를 출력해보기

In [ ]:
apple_spark_df.show(n=5)

#### 4> describe를 사용하여 데이터프레임의 컬럼별 통계보기

In [ ]:
apple_spark_df.describe().show()

#### 5> Close 컬럼의 평균값은 얼마인가?

In [ ]:
from pyspark.sql.functions import mean

apple_spark_df.select(mean("Close")).show()

#### 6> Volume 컬럼의 최대값과 최소값은?

In [ ]:
from pyspark.sql.functions import min, max

apple_spark_df.select(max("Volume"), min("Volume")).show()

#### 보너스 질문: HV ratio라는 이름의 새로운 컬럼을 추가한 데이터프레임을 만들기. 이 컬럼의 값은 High/Volume으로 계산된다

In [ ]:
apple_spark_df_with_hv = apple_spark_df.withColumn("hv ratio", apple_spark_df.High/apple_spark_df.Volume)

In [ ]:
apple_spark_df_with_hv.show(5)

#### 보너스 질문: 월별 Close 컬럼의 평균값은?

In [ ]:
from pyspark.sql.functions import month

monthdf = apple_spark_df.withColumn("Month", month("Date"))

In [ ]:
monthavgdf = monthdf.select(["Month", "Close"]).groupBy("Month").mean()

In [ ]:
monthavgdf.show()

In [ ]:
monthavgdf.select(["Month", "avg(Close)"]).orderBy("Month").show()